## Objectives of this lab

1. How to make calls to the foursquare api for different purposes
2. How to construct a URI to send request to the API to search 
    a. for specific types of venues.
    b. to explore a particular foursquare user.
    c. to explore a geographical location.
    d. to get trending venues around a location
3. how to use foilum vizualization lib

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [4]:

with open('creds.txt') as f:
    cred = f.readlines()

## Defining Foursquare credentials

In [7]:
CLIENT_ID = cred[0] # your Foursquare ID
CLIENT_SECRET = cred[1] # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Credentials loaded')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials loaded


### converting conrad hotel's address to latitude and longitudes

In [12]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

print(latitude, longitude)

40.7151482 -74.0156573


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

**Question: you want to search for Italian resteraunts in a close range to the conrad hotel**

In [13]:
search_query = 'Italian'
radius = 500
print(search_query)

Italian


**Define the corresponding URI**

In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                                   CLIENT_SECRET,
                                                                                                                                  latitude,
                                                                                                                                  longitude,
                                                                                                                                  VERSION,
                                                                                                                                  search_query,
                                                                                                                                  radius,
                                                                                                                                  LIMIT)
#print(url)

**Sending the get request and looking at the result**

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed14da298205d001bf23236'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 77,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15907755

In [24]:
results['response']['venues']

[{'id': '4fa862b3e4b0ebff2f749f06',
  'name': "Harry's Italian Pizza Bar",
  'location': {'address': '225 Murray St',
   'lat': 40.71521779064671,
   'lng': -74.01473940209351,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351},
    {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
   'distance': 77,
   'postalCode': '10282',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['225 Murray St',
    'New York, NY 10282',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1590775547',
  'hasPerk': False},
 {'id': '4f3232e219836c91c7bfde94',
  'name': 'Conca Cucina Italian Restaurant',
  'location': {'address': '63 W 

**Getting the relevant part of the json and converting it to a pandas DF**

In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4fa862b3e4b0ebff2f749f06,225 Murray St,US,New York,United States,77,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1590775547
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4f3232e219836c91c7bfde94,63 W Broadway,US,New York,United States,499,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,Conca Cucina Italian Restaurant,v-1590775547


**Define information of interest and Filter dataframe**

In [50]:
# keep only columns that include venue name, and anything that is associated with location

Filtered_columns = ['categories', 'name'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
Filtered_df = dataframe[Filtered_columns]

#function that extracts the category of the venue
def extract_cat(row):
    return(row['categories'][0]['name'])


Filtered_df['name'] = Filtered_df.apply(extract_cat, axis=1)

# clean column names by keeping only last term
Filtered_df.columns = [col.split('.')[-1] for col in Filtered_df.columns]
Filtered_df

/home/rishabh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,categories,name,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",Pizza Place,225 Murray St,US,New York,United States,77,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",Food,63 W Broadway,US,New York,United States,499,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,4f3232e219836c91c7bfde94


**Visualizing the Italian restraunt neaby**

In [65]:
venues_map  = folium.Map(location=[latitude, longitude], zoom_start=13)

#add ared circle marker to represent conrad hotel
folium.features.CircleMarker(location=[latitude, longitude],
                            radius=10,
                            color='red',
                            popup='Conrad Hotel',
                            fill_color='red',
                            fill=True,
                            fill_opacity=0.6).add_to(venues_map)


# add the Italian restaurants as blue circle markers

for lat, lng, label in zip(Filtered_df.lat, Filtered_df.lng, Filtered_df.name):
    folium.features.CircleMarker(location=[lat, lng],
                                radius=5,
                                color='blue',
                                popup = 'label',
                                fill_color='blue',
                                fill=True,
                                fill_opacity=0.6).add_to(venues_map)
    



venues_map